# Preparación de Dataset para Clasificación de Eventos Sísmicos

Este notebook prepara los datos para tareas de clasificación:

1. **Resampling:** Todas las señales a 200 Hz
2. **Estandarización de duración:** Todos los eventos con la misma longitud
3. **Padding:** Para eventos cortos (sin perder datos)
4. **Formato listo:** Arrays numpy para modelos de ML/DL
5. **División:** Train/Validation/Test

---

In [ ]:
from pathlib import Path
import yaml
import obspy
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy import signal
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Configuración
PROJECT_ROOT = Path.cwd().parent

with open(PROJECT_ROOT / "config.yaml", "r") as f:
    cfg = yaml.safe_load(f)

LABELED_DATA = Path(cfg["paths"]["labeled_data"])
FIGURES_DIR = PROJECT_ROOT / cfg["paths"]["figures_output"]
TABLES_DIR = PROJECT_ROOT / cfg["paths"]["tables_output"]

# Directorio para guardar dataset procesado
DATASET_DIR = PROJECT_ROOT / "processed_data"
DATASET_DIR.mkdir(exist_ok=True)

print("✓ Configuración cargada")
print(f"Datos origen: {LABELED_DATA}")
print(f"Dataset procesado: {DATASET_DIR}")

## 1. Exploración Inicial: Frecuencias y Duraciones

In [ ]:
print("Analizando archivos...\n")

event_info = []

for event_dir in LABELED_DATA.iterdir():
    if not event_dir.is_dir():
        continue
    
    event_type = event_dir.name
    mseed_files = [f for f in event_dir.iterdir() if f.is_file()]
    
    print(f"Procesando {event_type}... ", end="", flush=True)
    
    for mseed_file in mseed_files:
        try:
            st = obspy.read(str(mseed_file), headonly=True)
            tr = st[0]
            
            event_info.append({
                'tipo': event_type,
                'archivo': mseed_file.name,
                'filepath': str(mseed_file),
                'sampling_rate': tr.stats.sampling_rate,
                'npts': tr.stats.npts,
                'duracion_s': float(tr.stats.endtime - tr.stats.starttime)
            })
        except:
            continue
    
    print(f"✓ {len([e for e in event_info if e['tipo'] == event_type])} archivos")

df_info = pd.DataFrame(event_info)

print(f"\n{'='*60}")
print("RESUMEN DE DATOS")
print(f"{'='*60}")
print(f"Total de eventos: {len(df_info)}")
print(f"\nDistribución por tipo:")
print(df_info['tipo'].value_counts())

print(f"\nFrecuencias de muestreo encontradas:")
print(df_info['sampling_rate'].value_counts())

print(f"\nEstadísticas de duración:")
print(df_info.groupby('tipo')['duracion_s'].agg(['min', 'max', 'mean', 'std']))

# Encontrar duración máxima
max_duration = df_info['duracion_s'].max()
max_event = df_info.loc[df_info['duracion_s'].idxmax()]

print(f"\n{'='*60}")
print(f"DURACIÓN MÁXIMA: {max_duration:.2f} segundos")
print(f"Tipo: {max_event['tipo']}")
print(f"Archivo: {max_event['archivo']}")
print(f"{'='*60}")

## 2. Visualización de Distribuciones

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Duración por tipo
df_info.boxplot(column='duracion_s', by='tipo', ax=axes[0, 0])
axes[0, 0].set_title('Duración por Tipo de Evento', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Duración (s)')
axes[0, 0].get_figure().suptitle('')

# Histograma de duraciones
for event_type in df_info['tipo'].unique():
    subset = df_info[df_info['tipo'] == event_type]
    axes[0, 1].hist(subset['duracion_s'], alpha=0.6, label=event_type, bins=30)
axes[0, 1].axvline(max_duration, color='red', linestyle='--', linewidth=2, label=f'Max: {max_duration:.1f}s')
axes[0, 1].set_xlabel('Duración (s)')
axes[0, 1].set_ylabel('Frecuencia')
axes[0, 1].set_title('Distribución de Duraciones', fontsize=12, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Número de muestras por tipo
df_info.boxplot(column='npts', by='tipo', ax=axes[1, 0])
axes[1, 0].set_title('Número de Muestras por Tipo', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Número de muestras')
axes[1, 0].get_figure().suptitle('')

# Frecuencia de muestreo
sampling_counts = df_info.groupby(['tipo', 'sampling_rate']).size().unstack(fill_value=0)
sampling_counts.plot(kind='bar', ax=axes[1, 1], width=0.8)
axes[1, 1].set_title('Frecuencias de Muestreo por Tipo', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Cantidad de eventos')
axes[1, 1].set_xlabel('Tipo de Evento')
axes[1, 1].legend(title='Sampling Rate (Hz)')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(FIGURES_DIR / "dataset_info_duraciones.png", dpi=300, bbox_inches='tight')
print("✓ Gráfico guardado")
plt.show()

## 3. Configuración del Procesamiento

Definimos parámetros para la estandarización.

In [ ]:
# Parámetros de procesamiento
TARGET_SAMPLING_RATE = 200  # Hz
TARGET_DURATION = np.ceil(max_duration)  # Redondear hacia arriba
TARGET_SAMPLES = int(TARGET_SAMPLING_RATE * TARGET_DURATION)

print(f"{'='*60}")
print("PARÁMETROS DE PROCESAMIENTO")
print(f"{'='*60}")
print(f"Frecuencia objetivo: {TARGET_SAMPLING_RATE} Hz")
print(f"Duración objetivo: {TARGET_DURATION} segundos")
print(f"Número de muestras objetivo: {TARGET_SAMPLES}")
print(f"\nTodos los eventos serán procesados a esta configuración.")
print(f"Eventos más cortos: padding con ceros")
print(f"Eventos más largos: truncado (NO debería ocurrir)")
print(f"{'='*60}")

## 4. Funciones de Procesamiento

In [ ]:
def resample_trace(trace, target_sampling_rate):
    """
    Resamplea un trace a la frecuencia objetivo.
    
    Args:
        trace: ObsPy Trace
        target_sampling_rate: Frecuencia objetivo en Hz
    
    Returns:
        ObsPy Trace resampleado
    """
    current_rate = trace.stats.sampling_rate
    
    if current_rate == target_sampling_rate:
        return trace.copy()
    
    # Calcular factor de resampling
    if current_rate > target_sampling_rate:
        # Decimación
        factor = int(current_rate / target_sampling_rate)
        if current_rate / target_sampling_rate == factor:
            # Factor entero, usar decimate
            trace_copy = trace.copy()
            trace_copy.decimate(factor=factor, no_filter=False)
            return trace_copy
        else:
            # Factor no entero, usar resample
            trace_copy = trace.copy()
            trace_copy.resample(sampling_rate=target_sampling_rate)
            return trace_copy
    else:
        # Interpolación
        trace_copy = trace.copy()
        trace_copy.resample(sampling_rate=target_sampling_rate)
        return trace_copy


def standardize_length(data, target_length, padding_mode='constant'):
    """
    Estandariza la longitud de un array.
    
    Args:
        data: Array numpy
        target_length: Longitud objetivo
        padding_mode: 'constant' (ceros) o 'edge' (repetir último valor)
    
    Returns:
        Array de longitud target_length
    """
    current_length = len(data)
    
    if current_length == target_length:
        return data
    elif current_length < target_length:
        # Padding
        pad_width = target_length - current_length
        if padding_mode == 'constant':
            return np.pad(data, (0, pad_width), mode='constant', constant_values=0)
        elif padding_mode == 'edge':
            return np.pad(data, (0, pad_width), mode='edge')
    else:
        # Truncar (no debería pasar si TARGET_DURATION es el máximo)
        return data[:target_length]


def process_event(filepath, target_sampling_rate, target_samples):
    """
    Procesa un evento completo: resample + estandarización.
    
    Args:
        filepath: Ruta al archivo MSeed
        target_sampling_rate: Frecuencia objetivo
        target_samples: Número de muestras objetivo
    
    Returns:
        numpy array de forma (target_samples,) o None si hay error
    """
    try:
        # Leer archivo
        st = obspy.read(filepath)
        tr = st[0]  # Primer trace
        
        # Resample
        tr_resampled = resample_trace(tr, target_sampling_rate)
        
        # Estandarizar longitud
        data = tr_resampled.data.astype(np.float32)
        data_standardized = standardize_length(data, target_samples, padding_mode='constant')
        
        return data_standardized
    
    except Exception as e:
        return None


print("✓ Funciones de procesamiento definidas")

## 5. Procesamiento de Todos los Eventos

In [ ]:
print("Procesando eventos...\n")

X_data = []  # Señales
y_labels = []  # Etiquetas (tipo de evento)
metadata = []  # Información adicional

for idx, row in tqdm(df_info.iterrows(), total=len(df_info), desc="Procesando"):
    # Procesar evento
    signal_data = process_event(
        filepath=row['filepath'],
        target_sampling_rate=TARGET_SAMPLING_RATE,
        target_samples=TARGET_SAMPLES
    )
    
    if signal_data is not None:
        X_data.append(signal_data)
        y_labels.append(row['tipo'])
        
        metadata.append({
            'tipo': row['tipo'],
            'archivo': row['archivo'],
            'duracion_original': row['duracion_s'],
            'sampling_rate_original': row['sampling_rate']
        })

# Convertir a arrays numpy
X = np.array(X_data, dtype=np.float32)
y_str = np.array(y_labels)

# Encodear labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y_str)

# Metadata
metadata_df = pd.DataFrame(metadata)

print(f"\n{'='*60}")
print("DATASET PROCESADO")
print(f"{'='*60}")
print(f"Forma de X: {X.shape}")
print(f"  - {X.shape[0]} eventos")
print(f"  - {X.shape[1]} muestras por evento")
print(f"  - Duración: {X.shape[1] / TARGET_SAMPLING_RATE:.2f} segundos")
print(f"\nForma de y: {y.shape}")
print(f"\nClases:")
for i, class_name in enumerate(label_encoder.classes_):
    count = np.sum(y == i)
    print(f"  {i}: {class_name} ({count} eventos, {count/len(y)*100:.1f}%)")
print(f"\nTamaño en memoria: {X.nbytes / (1024**2):.2f} MB")
print(f"{'='*60}")

## 6. Normalización de Señales

Normalizamos cada señal individualmente (z-score normalization).

In [ ]:
def normalize_signals(X, method='zscore'):
    """
    Normaliza señales.
    
    Args:
        X: Array (n_samples, n_timesteps)
        method: 'zscore', 'minmax', o 'robust'
    
    Returns:
        X normalizado
    """
    X_norm = X.copy()
    
    if method == 'zscore':
        # Z-score por señal
        mean = X_norm.mean(axis=1, keepdims=True)
        std = X_norm.std(axis=1, keepdims=True) + 1e-8  # Evitar división por cero
        X_norm = (X_norm - mean) / std
    
    elif method == 'minmax':
        # Min-Max por señal
        min_val = X_norm.min(axis=1, keepdims=True)
        max_val = X_norm.max(axis=1, keepdims=True)
        X_norm = (X_norm - min_val) / (max_val - min_val + 1e-8)
    
    elif method == 'robust':
        # Robust scaling (mediana e IQR)
        median = np.median(X_norm, axis=1, keepdims=True)
        q75 = np.percentile(X_norm, 75, axis=1, keepdims=True)
        q25 = np.percentile(X_norm, 25, axis=1, keepdims=True)
        iqr = q75 - q25 + 1e-8
        X_norm = (X_norm - median) / iqr
    
    return X_norm


# Normalizar
X_normalized = normalize_signals(X, method='zscore')

print("✓ Señales normalizadas (z-score)")
print(f"  Media: {X_normalized.mean():.6f}")
print(f"  Std: {X_normalized.std():.6f}")
print(f"  Min: {X_normalized.min():.2f}")
print(f"  Max: {X_normalized.max():.2f}")

## 7. División Train/Validation/Test

In [ ]:
# División estratificada
# 70% train, 15% val, 15% test

# Primero: train+val (85%) vs test (15%)
X_temp, X_test, y_temp, y_test, meta_temp, meta_test = train_test_split(
    X_normalized, y, metadata_df,
    test_size=0.15,
    stratify=y,
    random_state=42
)

# Luego: train (70%) vs val (15%)
# 15% de 85% = 17.65% del temp
X_train, X_val, y_train, y_val, meta_train, meta_val = train_test_split(
    X_temp, y_temp, meta_temp,
    test_size=0.1765,
    stratify=y_temp,
    random_state=42
)

print(f"{'='*60}")
print("DIVISIÓN DEL DATASET")
print(f"{'='*60}")
print(f"\nTrain:")
print(f"  Shape: {X_train.shape}")
print(f"  Porcentaje: {len(X_train)/len(X)*100:.1f}%")
for i, class_name in enumerate(label_encoder.classes_):
    count = np.sum(y_train == i)
    print(f"    {class_name}: {count}")

print(f"\nValidation:")
print(f"  Shape: {X_val.shape}")
print(f"  Porcentaje: {len(X_val)/len(X)*100:.1f}%")
for i, class_name in enumerate(label_encoder.classes_):
    count = np.sum(y_val == i)
    print(f"    {class_name}: {count}")

print(f"\nTest:")
print(f"  Shape: {X_test.shape}")
print(f"  Porcentaje: {len(X_test)/len(X)*100:.1f}%")
for i, class_name in enumerate(label_encoder.classes_):
    count = np.sum(y_test == i)
    print(f"    {class_name}: {count}")

print(f"{'='*60}")

## 8. Visualización de Ejemplos Procesados

In [ ]:
# Seleccionar un ejemplo aleatorio de cada clase
fig, axes = plt.subplots(len(label_encoder.classes_), 2, figsize=(16, 3*len(label_encoder.classes_)))

fig.suptitle('Ejemplos de Señales Procesadas por Tipo', fontsize=16, fontweight='bold', y=0.995)

for i, class_name in enumerate(label_encoder.classes_):
    # Encontrar índice de un ejemplo de esta clase
    idx = np.where(y_train == i)[0][0]
    
    signal_data = X_train[idx]
    time = np.arange(len(signal_data)) / TARGET_SAMPLING_RATE
    
    # Dominio del tiempo
    axes[i, 0].plot(time, signal_data, linewidth=0.5)
    axes[i, 0].set_title(f'{class_name} - Dominio del Tiempo', fontsize=11, fontweight='bold')
    axes[i, 0].set_xlabel('Tiempo (s)')
    axes[i, 0].set_ylabel('Amplitud (normalizada)')
    axes[i, 0].grid(True, alpha=0.3)
    
    # Espectrograma
    f, t, Sxx = signal.spectrogram(signal_data, TARGET_SAMPLING_RATE, nperseg=256)
    im = axes[i, 1].pcolormesh(t, f, 10 * np.log10(Sxx + 1e-10), shading='gouraud', cmap='viridis')
    axes[i, 1].set_title(f'{class_name} - Espectrograma', fontsize=11, fontweight='bold')
    axes[i, 1].set_xlabel('Tiempo (s)')
    axes[i, 1].set_ylabel('Frecuencia (Hz)')
    axes[i, 1].set_ylim(0, 50)
    plt.colorbar(im, ax=axes[i, 1], label='Potencia (dB)')

plt.tight_layout()
plt.savefig(FIGURES_DIR / "ejemplos_senales_procesadas.png", dpi=300, bbox_inches='tight')
print("✓ Gráfico guardado")
plt.show()

## 9. Guardar Dataset Procesado

In [ ]:
import pickle

# Guardar en formato .npz (comprimido)
np.savez_compressed(
    DATASET_DIR / 'seismic_classification_dataset.npz',
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    sampling_rate=TARGET_SAMPLING_RATE,
    duration=TARGET_DURATION,
    n_samples=TARGET_SAMPLES
)

# Guardar label encoder
with open(DATASET_DIR / 'label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

# Guardar metadata
meta_train.to_csv(DATASET_DIR / 'metadata_train.csv', index=False)
meta_val.to_csv(DATASET_DIR / 'metadata_val.csv', index=False)
meta_test.to_csv(DATASET_DIR / 'metadata_test.csv', index=False)

# Guardar información del dataset
dataset_info = {
    'n_classes': len(label_encoder.classes_),
    'classes': label_encoder.classes_.tolist(),
    'n_train': len(X_train),
    'n_val': len(X_val),
    'n_test': len(X_test),
    'sampling_rate': TARGET_SAMPLING_RATE,
    'duration_seconds': float(TARGET_DURATION),
    'n_samples': TARGET_SAMPLES,
    'normalization': 'zscore',
    'train_split': 0.70,
    'val_split': 0.15,
    'test_split': 0.15
}

with open(DATASET_DIR / 'dataset_info.pkl', 'wb') as f:
    pickle.dump(dataset_info, f)

# También en JSON para lectura fácil
import json
with open(DATASET_DIR / 'dataset_info.json', 'w') as f:
    json.dump(dataset_info, f, indent=2)

print(f"{'='*60}")
print("ARCHIVOS GUARDADOS")
print(f"{'='*60}")
print(f"\nDirectorio: {DATASET_DIR}")
print(f"\nArchivos:")
print(f"  1. seismic_classification_dataset.npz (datos principales)")
print(f"  2. label_encoder.pkl (encoder de clases)")
print(f"  3. metadata_train/val/test.csv (metadatos)")
print(f"  4. dataset_info.json (información del dataset)")

# Calcular tamaño
total_size = sum((DATASET_DIR / f).stat().st_size 
                 for f in ['seismic_classification_dataset.npz', 
                          'label_encoder.pkl',
                          'metadata_train.csv',
                          'metadata_val.csv',
                          'metadata_test.csv'])

print(f"\nTamaño total: {total_size / (1024**2):.2f} MB")
print(f"{'='*60}")

## 10. Código para Cargar el Dataset

Ejemplo de cómo cargar los datos procesados en otro notebook o script.

In [ ]:
# CÓDIGO PARA CARGAR EL DATASET PROCESADO
# Copia esto en tu notebook de entrenamiento

load_example = '''
import numpy as np
import pickle
import json
from pathlib import Path

# Ruta al dataset
DATASET_DIR = Path("processed_data")

# Cargar datos
data = np.load(DATASET_DIR / "seismic_classification_dataset.npz")

X_train = data['X_train']
y_train = data['y_train']
X_val = data['X_val']
y_val = data['y_val']
X_test = data['X_test']
y_test = data['y_test']

# Cargar label encoder
with open(DATASET_DIR / 'label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

# Cargar info del dataset
with open(DATASET_DIR / 'dataset_info.json', 'r') as f:
    dataset_info = json.load(f)

print(f"Dataset cargado:")
print(f"  Train: {X_train.shape}")
print(f"  Val: {X_val.shape}")
print(f"  Test: {X_test.shape}")
print(f"  Clases: {dataset_info['classes']}")

# Para TensorFlow/Keras, agregar dimensión de canal
X_train_tf = X_train[..., np.newaxis]  # Shape: (n, timesteps, 1)
X_val_tf = X_val[..., np.newaxis]
X_test_tf = X_test[..., np.newaxis]

# Para PyTorch, cambiar orden de dimensiones
import torch
X_train_torch = torch.from_numpy(X_train).unsqueeze(1)  # Shape: (n, 1, timesteps)
y_train_torch = torch.from_numpy(y_train).long()
'''

print("EJEMPLO DE CÓDIGO PARA CARGAR EL DATASET:")
print("="*60)
print(load_example)
print("="*60)

## 11. Resumen Final

In [ ]:
print(f"\n{'='*60}")
print("✓ PROCESAMIENTO COMPLETADO")
print(f"{'='*60}")

print(f"\n📊 DATASET FINAL:")
print(f"  • {len(X)} eventos procesados")
print(f"  • {len(label_encoder.classes_)} clases: {', '.join(label_encoder.classes_)}")
print(f"  • Frecuencia: {TARGET_SAMPLING_RATE} Hz")
print(f"  • Duración: {TARGET_DURATION} segundos")
print(f"  • Muestras por evento: {TARGET_SAMPLES}")

print(f"\n📂 DIVISIÓN:")
print(f"  • Train: {len(X_train)} ({len(X_train)/len(X)*100:.1f}%)")
print(f"  • Validation: {len(X_val)} ({len(X_val)/len(X)*100:.1f}%)")
print(f"  • Test: {len(X_test)} ({len(X_test)/len(X)*100:.1f}%)")

print(f"\n✅ CARACTERÍSTICAS:")
print(f"  • Todas las señales a 200 Hz")
print(f"  • Duración estandarizada (padding con ceros)")
print(f"  • Normalización z-score por señal")
print(f"  • División estratificada")

print(f"\n💾 ARCHIVOS GUARDADOS EN: {DATASET_DIR}")

print(f"\n🚀 PRÓXIMOS PASOS:")
print(f"  1. Entrenar modelos baseline (RF, XGBoost)")
print(f"  2. Desarrollar modelos CNN-LSTM")
print(f"  3. Evaluar con X_test y y_test")

print(f"\n{'='*60}")